# AI-Assisted Multi-Reference Timekeeping Overview

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-org/ai-multi-reference-timekeeping/blob/main/notebooks/00_overview.ipynb)

This notebook validates that the time server scaffold can fuse sensor data with reference measurements.

In [ ]:
# Clone and install the repo when running in Colab
import os
if not os.path.exists('ai-multi-reference-timekeeping'):
    !git clone https://github.com/your-org/ai-multi-reference-timekeeping.git
%cd ai-multi-reference-timekeeping
!pip install -e .


In [ ]:
from ai_multi_reference_timekeeping.fusion import HeuristicFusion, VirtualClock
from ai_multi_reference_timekeeping.kalman import ClockCovariance, ClockKalmanFilter, ClockState
from ai_multi_reference_timekeeping.time_server import AudioFeatureSensor, MlVarianceModel, SensorAggregator, TimeServer

class DummyAudio:
    def sample(self):
        return [0.01] * 256, 8000

kalman = ClockKalmanFilter(
    state=ClockState(offset=0.0, drift=0.0),
    covariance=ClockCovariance(p00=1.0, p01=0.0, p10=0.0, p11=1.0),
    process_noise_offset=1e-4,
    process_noise_drift=1e-6,
)
clock = VirtualClock(kalman_filter=kalman, fusion=HeuristicFusion())

class StaticReference:
    def sample(self, frame):
        from ai_multi_reference_timekeeping.fusion import Measurement
        return Measurement(name='ref', offset=0.15, variance=0.2)

server = TimeServer(
    clock=clock,
    references=[StaticReference()],
    sensors=SensorAggregator(AudioFeatureSensor(DummyAudio())),
    inference=MlVarianceModel(feature_weights={'ambient_audio_db': 0.01}),
)
update, frame, drift, drift_hint = server.step(1.0)
print(update.fused_offset, drift.drift, drift_hint)
